<a href="https://colab.research.google.com/github/WafaSanaa/Computer_Vision/blob/main/Classification_(Cancer_de_poumons).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

In [2]:
pip install numpy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install matplotlib

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
data = ("/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data ")

In [7]:
train_path_str = '/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data/train'

In [8]:
val_path_str = '/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data/valid'

In [9]:
test_path_str = '/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data/test'

In [10]:
import os
test_classes = os.listdir(test_path_str)

# **3. Pré-proccessing Data**

In [11]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout
#Conv2D: couche de convolution 2D pour extraire des caracteristiques des images
#Dense : couche de cnx
#MaxPooling2D : couche de sous echantillonage qui reduit les dimensions
#Flatten : couche qui aplatit les données en entrées en un vecteur
#BatchNormalization : couche qui normalise les activations de la couche précédente
#Dropout : couche de régularisation pour prévenir le sur apprentissage
from sklearn.utils import shuffle
#fonction pour mélanger les ensembles de données

In [12]:
from keras.utils import to_categorical
#to_categorical : fonction pour convertir des vecteurs de classes en matrices de classes binaires

In [13]:
from sklearn.metrics import confusion_matrix, classification_report
# fonctions pour évaluer les modèles de classification

In [14]:
#def du nombre de classe de sortie
num_classes = 4

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:

from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
#configuration du générateur d'image pour l'entrainement avec augmentation de données
trainGenerator = ImageDataGenerator(
    preprocessing_function = preprocess_input , #prétraitement des image
    rotation_range = 10,  # rotation aléatoire des images dans une plage de 10 degrés
    width_shift_range = 0.3,  # translation horizontale aléatoire des images dans une plage de 30%
    height_shift_range = 0.3,  # translation verticale aléatoire des images dans une plage de 30%
    shear_range = 0.2,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    dtype = 'float32'
)

In [34]:
valGenerator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)

In [35]:
testGenerator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)

In [36]:
train_data = trainGenerator.flow_from_directory(
    train_path_str,                   # chemin du dossier contenant les images d'entraînement
    target_size = (224, 224),         # taille des images en entrée du modèle
    batch_size = 16,                  # nombre d’images par lot (batch)
    class_mode = 'categorical'        # pour la classification multi-classes
)

Found 613 images belonging to 4 classes.


In [37]:
val_data = valGenerator.flow_from_directory(
    val_path_str,                     # chemin du dossier de validation
    target_size = (224, 224),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 72 images belonging to 4 classes.


In [38]:
test_data = testGenerator.flow_from_directory(
    test_path_str,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 315 images belonging to 4 classes.


In [40]:
input_shape = (224, 224, 3)
VGG16_model = VGG16 (
    include_top = False,
    weights = "imagenet" ,
    input_shape = input_shape
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [41]:
for layer in VGG16_model.layers:
    layer.trainable = False

## **Création de Modèle**

In [48]:
from keras import activations
from tensorflow.keras.models import Sequential # Import Sequential from tensorflow.keras.models

#création d'un modèle séquentiel
model = Sequential()

#Ajout du modèle vgg16 pré-entrainé sans la couche fully connected
model.add(VGG16_model)

# Ajout d'une couche de normalisation par lots pour améliorer la stabilité et la vitesse d'apprentissage
model.add(BatchNormalization())

# Ajout d'une couche de pooling max pour réduire la dimensionnalité des données
model.add(MaxPooling2D(2, 2))

# Aplatissement des données pour les préparer à l'entrée des couches Fully connected
model.add(Flatten())

# Première couche fully connected avec 1024 neurones activés par la fonction ReLU
model.add(Dense(1024, activation='relu'))

# Dropout avec un taux de 30% pour prévenir le surapprentisage
model.add(Dropout(0.3))

# deuxieme couche fully connected avec 1024 neurones activés par la fonction ReLU
model.add(Dense(512, activation='relu'))

# Dropout avec un taux de 50% pour prévenir le surapprentisage
model.add(Dropout(0.5))

